# Лабораторная работа 7, Лохматов Никита Игоревич М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для семантической сегментации: product-masks-sample 

Этот набор данных включает в себя изображения, созданные на основе 3D-моделей предметов домашнего интерьера и гостиной обстановки.

Особенностью данного датасета является фотореалистичное исполнение изображений, что обеспечивает высокую степень достоверности при использовании их для обучения компьютерных моделей. Благодаря этому, результаты сегментации получаются максимально приближенными к работе с реальными фотографиями.

Подобные данные представляют особую ценность для разработки систем автоматизированного учета и инвентаризации, где требуется точное распознавание объектов через камеры видеонаблюдения или мобильные устройства. Возможность работы с синтетическими, но при этом реалистичными изображениями значительно упрощает процесс обучения моделей, позволяя избежать трудоемкого сбора и разметки реальных фотографий.

### b. Выбор метрик качества и обоснование

Для задачи семантической сегментации ключевым аспектом является выбор метрик, которые корректно отражают качество предсказаний с учетом специфики задачи. Здесь важно учитывать многоклассовую классификацию, пространственную структуру объектов и возможный дисбаланс классов.

В качестве основной метрики была выбрана IoU (Intersection over Union) — коэффициент, вычисляемый как отношение площади пересечения предсказанной и истинной масок к площади их объединения для каждого класса. Эта метрика хорошо отражает точность локализации объектов и учитывает их форму, что особенно важно в семантической сегментации.

В отличие от задач классификации, стандартная Accuracy здесь часто оказывается неинформативной. Это связано с тем, что изображения могут содержать доминирующие классы (например, фон), которые занимают большую часть сцены. В таком случае модель может достигать высокой Accuracy, просто предсказывая везде фоновый класс, при этом плохо распознавая остальные объекты. По этой причине Accuracy не была использована для оценки модели.

## 2. Создание бейзлайна и оценка качества

### a. Обучение моделей

Импортируем библиотеки

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import datasets
import torch
import numpy as np
import segmentation_models_pytorch as smp
import torchinfo
import matplotlib.pyplot as plt
import albumentations
import os
import torchvision

from PIL import Image
from datasets import load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Скачиваем датасет

In [51]:
# Увеличиваем таймауты для загрузки датасета
os.environ["HF_DATASETS_TIMEOUT"] = "600"

# Скачиваем датасет
dataset = datasets.load_dataset('Nfiniteai/product-masks-sample')

Формируем бейзлайн. Нормализуем название классов и уберём из тренировочной выборки те записи, в которых есть классы, не предусмотренные в валидационной выборке


In [52]:
# Делим датасет на валидационный и тренировочный
val_dataset = dataset['val']
train_dataset = dataset['train']

# Обработка валидационного набора
val_dataset = val_dataset.map(
    lambda batch: {
        'category': [x.lower().replace(' ', '_') for x in batch['category']]
    },
    batched=True,
    batch_size=300,
    writer_batch_size=300,
    load_from_cache_file=False
)

# Обработка тренировочного набора
train_dataset = train_dataset.map(
    lambda batch: {
        'category': [x.lower().replace(' ', '_') for x in batch['category']]
    },
    batched=True,
    batch_size=300,
    writer_batch_size=300,
    load_from_cache_file=False
)

# Фильтруем датасет
train_dataset = train_dataset.remove_columns(set(train_dataset.features) - {
    'image',
    'bbox',
    'category',
    'mask'
})

val_dataset = val_dataset.remove_columns(set(train_dataset.features) - {
    'image',
    'bbox',
    'category',
    'mask',
})

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Для корректного обучения модели сначала исключим из тренировочных данных классы, отсутствующие в валидационной выборке, и проверим, что не удалили слишком много примеров. Затем каждому классу назначим уникальный числовой ID, добавив специальный идентификатор <back> для фона.

Отдельное внимание уделим обработке масок: функция transform_mask конвертирует черно-белую маску в матрицу числовых идентификаторов классов, где фон помечается ID класса <back>. Это обеспечит корректную подготовку данных для обучения модели.

In [53]:
new_class = '<back>'

categories = set(val_dataset['category'])
categories = {category: index for index, category in enumerate(categories)}
categories[new_class] = len(categories)

train_dataset = train_dataset.filter(
    lambda category: category in categories,
    load_from_cache_file=False,
    writer_batch_size=300,
    input_columns=['category'],
)

transform_image = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


def transform_mask(mask, class_index, *, threshold=0.5, classes_amount=1):
    mask = mask.resize((224, 224))
    mask_np = np.array(mask).astype(np.float32) / 255.0

    object_mask = (mask_np >= threshold).astype(int) * class_index
    background_mask = (mask_np < threshold).astype(int) * categories[new_class]
    total_mask = object_mask + background_mask

    return torch.from_numpy(total_mask)

def default_transform(x, bbox, y):
    return x, y

class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, categories, *, max_len=None, transform=None, threshold=0.5):
        self._dataset = dataset
        self._categories = categories
        self._max_len = float('inf') if max_len is None else max_len
        self._threshold = threshold
        self._transform = transform or default_transform

    def __len__(self):
        return min(len(self._dataset), self._max_len)

    def __getitem__(self, idx):
        example = self._dataset[idx]
        image, mask = example['image'].convert('RGB'), example['mask']
        class_index = self._categories[example['category']]
        bbox = example['bbox']

        image, mask = self._transform(image, bbox, mask)

        image = transform_image(image)
        mask = transform_mask(
            mask,
            class_index,
            threshold=self._threshold,
            classes_amount=len(self._categories),
        )

        return image, mask

Filter:   0%|          | 0/2559 [00:00<?, ? examples/s]

Также, для визуальной проверки работоспособности обученной модели напишем функцию test_model, которая визуально отображает маску, предсказанную моделью, и ожидаемую маску для класса category_index, объект которого находится на целевом изображении

In [54]:
def _calculate_metrics(outputs, masks, metrics):
    predicted_classes = outputs.argmax(dim=1)
    tp, fp, fn, tn = smp.metrics.get_stats(
        predicted_classes, masks,
        mode='multiclass',
        num_classes=len(categories) - 1,
        ignore_index=categories[new_class],
    )

    return {
        metric_name: metric_function(tp, fp, fn, tn).item()
        for metric_name, metric_function in metrics.items()
    }


def train_model(model, dataset, loss, optimizer, *, num_epochs=1):
    model.train()

    total_loss = 0.0
    total_size = 0

    for epoch in range(1, num_epochs + 1):
        for step_number, (images, masks) in enumerate(dataset, 1):
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss_value = loss(outputs, masks)
            loss_value.backward()
            optimizer.step()

            batch_size = images.size()[0]
            total_loss += loss_value.item() * batch_size
            total_size += batch_size

            yield {
                'epoch': epoch,
                'step': step_number,
                'loss': total_loss / total_size,
            }


def eval_model(model, ds, metrics):
    model.eval()

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, masks in ds:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)

            all_labels.append(masks.cpu())
            all_preds.append(outputs.cpu())

    all_labels = torch.cat(all_labels, dim=0)
    all_preds  = torch.cat(all_preds, dim=0)

    return _calculate_metrics(all_preds, all_labels, metrics)

def test_model(model, image, mask, category_index):
    model.eval()
    with torch.no_grad():
        C, W, H = image.shape
        image = image.resize(1, C, W, H)
        image = image.to(device)
        preds = model(image)[0].cpu()

    predicted_mask = torch.argmax(preds, dim=0).numpy()
    predicted_mask = (predicted_mask == category_index).astype(np.uint8) * 255

    predicted_mask_image = Image.fromarray(predicted_mask, mode='L')
    mask_image = Image.fromarray((mask.numpy() == category_index).astype(np.uint8) * 255, mode='L')

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(mask_image)

    plt.subplot(1, 2, 2)
    plt.imshow(predicted_mask_image)

    plt.show()

# Задаём метрики
metrics = {
    'iou': lambda tp, fp, fn, tn: smp.metrics.iou_score(tp, fp, fn, tn, reduction='micro'),
}

Обучение модели CNN

In [55]:
model_cnn = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
)

train_ds = SegmentationDataset(train_dataset, categories)
val_ds = SegmentationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model_cnn = model_cnn.to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model_cnn,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model_cnn.parameters(), lr=1e-3),
    )

    for i, log in enumerate(train_logs):
        if i % 10 == 0:
            print(f'Loss: {log["loss"]}')


Epoch: 1
Loss: 3.4138243198394775
Loss: 2.9628832123496314
Loss: 3.218384481611706
Loss: 2.97334647563196
Loss: 2.951845012060026
Loss: 2.8129118564082125
Loss: 2.8080450042349394
Loss: 2.764828304169883
Loss: 2.843307321454272
Loss: 2.8291472576476715
Loss: 2.8225717119651264
Loss: 2.7533656970874683
Loss: 2.7264289796844987
Epoch: 2
Loss: 1.8158044815063477
Loss: 1.9192100167274475
Loss: 2.383252271584102
Loss: 2.2137862751560826
Loss: 2.2687896693625103
Loss: 2.180067342870376
Loss: 2.2766229168313448
Loss: 2.2886755751891874
Loss: 2.419335310841784
Loss: 2.4616209949765886
Loss: 2.4821709783950654
Loss: 2.455619841008573
Loss: 2.4502612362223224
Epoch: 3
Loss: 1.7126054763793945
Loss: 1.8325958035208962
Loss: 2.4014258952367875
Loss: 2.334663268058531
Loss: 2.3277903242809015
Loss: 2.243458205578374
Loss: 2.324028146071512
Loss: 2.3340043601855425
Loss: 2.444059279229906
Loss: 2.4913194192634833
Loss: 2.504410711845549
Loss: 2.4632310878049144
Loss: 2.4410962624983354


Обучение трансформерной модели

In [56]:
model_transform = smp.Segformer(
    encoder_name='mit_b0',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
)

torchinfo.summary(model_transform)

train_ds = SegmentationDataset(train_dataset, categories)
val_ds = SegmentationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model_transform = model_transform.to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model_transform,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model_transform.parameters(), lr=1e-3),
    )

    for i, log in enumerate(train_logs):
        if i % 10 == 0:
         print(f'Loss: {log["loss"]}')

Epoch: 1
Loss: 3.9942052364349365
Loss: 2.9918160980398003
Loss: 3.2698717628206526
Loss: 3.073573416279208
Loss: 3.0041509168904
Loss: 2.846421241760254
Loss: 2.860645837471133
Loss: 2.8707947059416434
Loss: 2.9498332731517745
Loss: 2.9553522572412594
Loss: 2.957405154657836
Loss: 2.891308440281464
Loss: 2.8535280203031115
Epoch: 2
Loss: 1.3732373714447021
Loss: 2.0182564854621887
Loss: 2.5124746759732566
Loss: 2.3883527921092127
Loss: 2.3510602087509342
Loss: 2.2407586352497924
Loss: 2.350883430144826
Loss: 2.3755307357076187
Loss: 2.474949660860462
Loss: 2.5039859479600257
Loss: 2.5218013259443905
Loss: 2.4774803100405514
Loss: 2.453173875808716
Epoch: 3
Loss: 1.3790245056152344
Loss: 1.768868793140758
Loss: 2.2981909854071483
Loss: 2.165590968824202
Loss: 2.1461121527160087
Loss: 2.0417541651164783
Loss: 2.138639597619166
Loss: 2.1959176021562494
Loss: 2.315572620174031
Loss: 2.3463865289321313
Loss: 2.401363742823648
Loss: 2.393601336457708
Loss: 2.3880639494943225


### b. Оценка качества моделей по выбранным метрикам на выбранном наборе данных

CNN

In [57]:
results_cnn = eval_model(model_cnn, val_loader, metrics)

print('CNN res: {}'.format(
    ', '.join(f"{name}:{value:.4f}" for name, value in results_cnn.items())
))

CNN res: iou:0.0591


Трансформерная модель

In [58]:
results_transform = eval_model(model_transform, val_loader, metrics)

print('Transform res: {}'.format(
    ', '.join(f"{name}:{value:.4f}" for name, value in results_transform.items())
))

Transform res: iou:0.1348


## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы (аугментации данных, подбор моделей, подбор гиперпараметров и т.д.)


1. Обрезка изображений

В текущем бейзлайне основную часть изображения занимает задний фон, и доля пикселей объекта становится слишком малой. Чтобы повысить информативность данных, исключим фон, обрезав изображения и маски по bounding box — так на кадре останется только целевой объект. Это увеличит относительную долю полезных пикселей и, вероятно, улучшит обучение модели. Для реализации создадим функцию crop_image, выполняющую обрезку по bbox, и интегрируем её в датасеты.

2. Аугментация

Вторая гипотеза — применение аугментаций для повышения обобщающей способности модели. Для этого с помощью albumentations реализуем пайплайн _augment, включающий случайное горизонтальное отражение и вращение до 15°. Благодаря albumentations, одинаковые преобразования будут применяться как к изображению, так и к маске.

### b. Проверка гипотез

1. Обрезка изображений

In [59]:
def crop_image(image, bbox, mask):
    x, y, w, h = bbox
    cropped_image = image.crop((x, y, x + w, y + h))
    cropped_mask = mask.crop((x, y, x + w, y + h))
    return cropped_image, cropped_mask


train_ds = SegmentationDataset(train_dataset, categories, transform=crop_image)
val_ds = SegmentationDataset(val_dataset, categories, transform=crop_image)

In [60]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('CNN crop res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
CNN crop res: iou:0.0080
Epoch: 2
CNN crop res: iou:0.0080
Epoch: 3
CNN crop res: iou:0.0080


In [61]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = smp.Segformer(
    encoder_name='mit_b0',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)
    
    print('Transform crop res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))


Epoch: 1
Transform crop res: iou:0.0210
Epoch: 2
Transform crop res: iou:0.0210
Epoch: 3
Transform crop res: iou:0.0210


2. Аугментация

In [62]:
_augment = albumentations.Compose(
    [
        albumentations.HorizontalFlip(p=0.5),
        albumentations.Rotate(limit=15, p=0.5),
    ],
    additional_targets={'mask': 'mask'},
)


def augment_pair(image, mask):
    image = np.array(image)
    mask = np.array(mask)

    augmented = _augment(image=image, mask=mask)
    return Image.fromarray(augmented['image']), Image.fromarray(augmented['mask'])


def transform(image, bbox, mask):
    image, mask = augment_pair(image, mask)
    return crop_image(image, bbox, mask)


train_ds = SegmentationDataset(train_dataset, categories, transform=transform)
val_ds = SegmentationDataset(val_dataset, categories, transform=crop_image)

In [63]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('CNN aug res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
CNN aug res: iou:0.0131
Epoch: 2
CNN aug res: iou:0.0131
Epoch: 3
CNN aug res: iou:0.0131


In [64]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = smp.Segformer(
    encoder_name='mit_b0',
    encoder_weights='imagenet',
    in_channels=3,
    classes=len(categories),
).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('transform aug res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
transform aug res: iou:0.0152
Epoch: 2
transform aug res: iou:0.0152
Epoch: 3
transform aug res: iou:0.0152


### g. Выводы

По результатам обучения базовых моделей для задачи семантической сегментации можно отметить следующее. На второй эпохе наблюдается просадка IoU, вероятно связанная с адаптацией архитектур к задаче. SegFormer на MiT-B0 обеспечил более высокое качество сегментации и стабильное обучение по сравнению с U-Net на ResNet18.

Обрезка изображений по bounding box и аугментации улучшили результаты лишь для U-Net, тогда как у SegFormer качество, наоборот, слегка снизилось. Таким образом, предложенные методы усиления бейзлайна эффективны преимущественно для сверточных моделей, в то время как на трансформерной архитектуре SegFormer значимого улучшения не наблюдается.

## 4. Имплементация алгоритма машинного обучения 

### a. Самостоятельная имплементация алгоритмов машинного обучения для классификации и регрессии

**Свёрточная модель**

Перейдём к реализации сверточной модели для семантической сегментации — U-Net. В этот раз архитектура оформлена в виде класса UNetImplementation с гибкой конфигурацией уровней через словари, что позволяет легко адаптировать модель под разные задачи и ресурсы.

В конструктор передаются параметры каналов для каждого уровня (in_channels, out_channels) и число классов (num_classes) для финального слоя.
Модель состоит из энкодера (экстракция признаков с понижением масштаба), центрального боттлнека (максимальное сжатие и увеличение глубины), декодера (апсемплирование и skip-соединения) и финального свёрточного слоя, возвращающего маску предсказаний.
Метод forward реализует пошаговую обработку данных: формирование признаков в энкодере, skip-соединения, объединение признаков при декодировании и получение итоговой маски.



In [65]:
class UNetImplementation(torch.nn.Module):
    def __init__(self, layers_config, num_classes):
        super().__init__()

        self._encoder_blocks = torch.nn.ModuleList()
        self._downsampling_layers = torch.nn.ModuleList()

        for layer in layers_config:
            self._encoder_blocks.append(
                torch.nn.Sequential(
                    torch.nn.Conv2d(
                        layer['in_channels'],
                        layer['out_channels'],
                        kernel_size=3,
                        padding=1,
                    ),
                    torch.nn.ReLU(),
                    torch.nn.Conv2d(
                        layer['out_channels'],
                        layer['out_channels'],
                        kernel_size=3,
                        padding=1,
                    ),
                    torch.nn.ReLU(),
                )
            )
            self._downsampling_layers.append(torch.nn.MaxPool2d(kernel_size=2))

        bottleneck_in = layers_config[-1]['out_channels']
        bottleneck_out = bottleneck_in * 2

        self._bottleneck_block = torch.nn.Sequential(
            torch.nn.Conv2d(
                bottleneck_in,
                bottleneck_out,
                kernel_size=3,
                padding=1,
            ),
            torch.nn.ReLU(),
            torch.nn.Conv2d(
                bottleneck_out,
                bottleneck_out,
                kernel_size=3,
                padding=1,
            ),
            torch.nn.ReLU(),
        )

        self._upsampling_layers = torch.nn.ModuleList()
        self._decoder_blocks = torch.nn.ModuleList()
        reversed_config = list(reversed(layers_config))

        for layer in reversed_config:
            self._upsampling_layers.append(
                torch.nn.ConvTranspose2d(
                    bottleneck_out,
                    layer['out_channels'],
                    kernel_size=2,
                    stride=2,
                ),
            )

            self._decoder_blocks.append(
                torch.nn.Sequential(
                    torch.nn.Conv2d(
                        layer['out_channels'] * 2,
                        layer['out_channels'],
                        kernel_size=3,
                        padding=1,
                    ),
                    torch.nn.ReLU(),
                    torch.nn.Conv2d(
                        layer['out_channels'],
                        layer['out_channels'],
                        kernel_size=3,
                        padding=1,
                    ),
                    torch.nn.ReLU(),
                )
            )

            bottleneck_out = layer['out_channels']

        self._final_convolution = torch.nn.Conv2d(
            layers_config[0]['out_channels'],
            num_classes,
            kernel_size=1,
        )

    def forward(self, input_tensor):
        encoder_feature_maps = []
        x = input_tensor

        for encoder_block, downsample in zip(self._encoder_blocks, self._downsampling_layers):
            x = encoder_block(x)
            encoder_feature_maps.append(x)
            x = downsample(x)

        x = self._bottleneck_block(x)

        for index in range(len(self._upsampling_layers)):
            x = self._upsampling_layers[index](x)
            skip_connection = encoder_feature_maps[-(index + 1)]
            x = self._decoder_blocks[index](torch.cat([x, skip_connection], dim=1))

        return self._final_convolution(x)

**Трансформерная модель**

Перейдём к собственной реализации трансформерной модели для семантической сегментации, оформленной в виде классов. Архитектура включает ключевые компоненты:

- PatchEmbedding — разбивает изображение на патчи и проецирует их в скрытое пространство через свёртку;
- TransformerEncoderBlock — состоит из слоёв нормализации, multi-head attention и двухслойного MLP с GELU и дропаутом, с резидуальными связями для стабильности обучения;
- TransformerSegmentationModel — объединяет все модули в единую модель сегментации.

In [66]:
class PatchEmbedding(torch.nn.Module):
    def __init__(self, in_channels=3, patch_size=16, emb_dim=256):
        super().__init__()
        self.patch_size = patch_size
        self.projection = torch.nn.Conv2d(
            in_channels,
            emb_dim,
            kernel_size=patch_size,
            stride=patch_size,
        )

    def forward(self, x):
        x = self.projection(x)
        batch_size, channels, height, width = x.shape
        x = x.flatten(2).transpose(1, 2)

        return x


class TransformerEncoderBlock(torch.nn.Module):
    def __init__(self, emb_dim=256, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()

        self.layer_norms = torch.nn.ModuleList([
            torch.nn.LayerNorm(emb_dim),
            torch.nn.LayerNorm(emb_dim)
        ])

        self.attention = torch.nn.MultiheadAttention(
            emb_dim,
            num_heads,
            dropout=dropout,
            batch_first=True,
        )

        hidden_dim = int(emb_dim * mlp_ratio)

        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(emb_dim, hidden_dim),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim, emb_dim),
            torch.nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attention(self.layer_norms[0](x), x, x)[0]
        x = x + self.mlp(self.layer_norms[1](x))

        return x


class TransformerSegmentationModel(torch.nn.Module):
    def __init__(
        self,
        in_channels=3,
        num_classes=1,
        patch_size=16,
        emb_dim=256,
        depth=6,
        num_heads=8,
    ):
        super().__init__()

        self.patch_embed = PatchEmbedding(in_channels, patch_size, emb_dim)

        self.blocks = torch.nn.ModuleList([
            TransformerEncoderBlock(emb_dim, num_heads)
            for _ in range(depth)
        ])

        num_patches = (224 // patch_size) ** 2
        self.position_embedding = torch.nn.Parameter(
            torch.zeros(1, num_patches, emb_dim)
        )

        self.reconstruction = torch.nn.ConvTranspose2d(
            emb_dim,
            num_classes,
            kernel_size=patch_size,
            stride=patch_size,
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.patch_embed(x)
        x = x + self.position_embedding

        for block in self.blocks:
            x = block(x)

        spatial_dim = int((x.size(1)) ** 0.5)
        x = x.transpose(1, 2).reshape(batch_size, -1, spatial_dim, spatial_dim)
        x = self.reconstruction(x)

        return x

### b. Обучение имплементированной модели

Свёрточная

In [67]:
train_ds = SegmentationDataset(train_dataset, categories)
val_ds = SegmentationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

config = [
    {'in_channels': 3, 'out_channels': 64},
    {'in_channels': 64, 'out_channels': 128},
]
model = UNetImplementation(config, num_classes=len(categories)).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)
    
    print('Res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
Res: iou:0.0002
Epoch: 2
Res: iou:0.0002
Epoch: 3
Res: iou:0.0002


Трансформерная

In [68]:
train_ds = SegmentationDataset(train_dataset, categories)
val_ds = SegmentationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = TransformerSegmentationModel(num_classes=len(categories)).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('Res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
Res: iou:0.0128
Epoch: 2
Res: iou:0.0128
Epoch: 3
Res: iou:0.0128


### g. Обучение на улучшенном бейзлайне

Свёрточная

In [69]:
train_ds = SegmentationDataset(train_dataset, categories, transform=transform)
val_ds = SegmentationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = UNetImplementation(config, num_classes=len(categories)).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')
    
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('Res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
Res: iou:0.0091
Epoch: 2
Res: iou:0.0091
Epoch: 3
Res: iou:0.0091


Трансформерная

In [70]:
train_ds = SegmentationDataset(train_dataset, categories, transform=transform)
val_ds = SegmentationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

model = TransformerSegmentationModel(num_classes=len(categories)).to(device)

for epoch in range(3):
    print(f'Epoch: {epoch + 1}')

    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(ignore_index=categories[new_class]),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    results = eval_model(model, val_loader, metrics)

    print('Res: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Epoch: 1
Res: iou:0.0121
Epoch: 2
Res: iou:0.0121
Epoch: 3
Res: iou:0.0121


### j. Выводы

Собственная трансформерная модель показала худшие результаты на бейзлайне по сравнению с U-Net, а улучшение бейзлайна существенного эффекта не дало.
В
озможные причины — отсутствие предобучения и меньшая сложность архитектуры по сравнению с U-Net и SegFormer, обученными на больших датасетах.

Также собственная реализация уступила по качеству предобученному SegFormer, вероятно, из-за упрощённой структуры и обучения с нуля.

Однако применение улучшенного бейзлайна в этом случае дало заметный прирост метрик по сравнению с обычным бейзлайном.